# Rules Engines Examples

This notebook shows how to use rules engines in a simple case: just execute the reasoning defined in a rules file over a set of data and get the results.

In [1]:
import sys

# Add code folder to the execution path
PYTHON_PATH = ["../src",
               "../examples",
               "."]
for p in PYTHON_PATH:
    if p not in sys.path:
        sys.path.append(p)

In [2]:
from auracog_rules.rules_engines import RulesEngine, RulesEnginePool
from auracog_rules.rules_engines_persistence import RulesEnginesStore, LocalFileRulesEngineStore

import logging
import time

Create a directory for the rules states store.

In [3]:
!mkdir -p rules_store

Initialize logging.

In [4]:
logging.basicConfig(format="[%(levelname)s] %(asctime)s.%(msecs)03d - %(name)s: %(message)s",
                        datefmt="%d-%m-%Y %H:%M:%S",
                        level=logging.DEBUG)
logger = logging.getLogger(__name__)


Create a local store to save rules engines states. 

This is an __optional__ step. If no persistent state needs to be stored use `store=None` (or simply do not include this parameter, since it is not mandatory)

In [5]:
rules_persistence_store = LocalFileRulesEngineStore("rules_store", binary=False)

- Create a pool of rules engines. Load the rules and the rest of the CLIPS code to be used. Multiple files can be loaded. Take into account that they are loaded in the same order as declared.

- Also define define the name of a __python library__ (it should be accesible through the system path) containing all the python functions that will be used in the clips code. In this example see [auracog_suggestions.custom_functions](../examplesauracog_suggestions/custom_functions.py).

- The __pool size__ defines the number of rules engines to be kept in the pool. If the number of acuired rules engiens at a given instant exceeds this size, the acquiring operation will wait until one is available.
- The __initial pool size__ defines the number or rules engines that will be pre-loaded at pool instantiation time. All the rule engines not pre-loaded will be created and intialized on demmand (within the limits defined by the __pool size__).


Execute `help(RulesEnginePool)`for further information.


In [6]:
POOL_NAME = "test_pool"  # This is the name of the rules engine pool. Just for informative purposes.
RULES_FILES = ["../clips_modules/slots_control.clp", 
               "example_suggestions.clp"]
#FUNCTIONS_PACKAGE_NAME = "auracog_suggestions.custom_functions"
FUNCTIONS_PACKAGE_NAME = "custom_functions"
POOL_SIZE = 2
INITIAL_POOL_SIZE = 2
rules_engine_pool = RulesEnginePool(POOL_NAME, RULES_FILES, 
                                    functions_package_name=FUNCTIONS_PACKAGE_NAME,
                                    pool_size=POOL_SIZE, 
                                    initial_pool_size=INITIAL_POOL_SIZE, 
                                    store=rules_persistence_store)

[INFO] 19-11-2019 11:11:16.372 - auracog_rules.rules_engines: Defining function get_time
[INFO] 19-11-2019 11:11:16.401 - auracog_rules.rules_engines: Rules engine created in 46.324 ms
[INFO] 19-11-2019 11:11:16.407 - auracog_rules.rules_engines: Defining function get_time
[INFO] 19-11-2019 11:11:16.420 - auracog_rules.rules_engines: Rules engine created in 16.669 ms


__Acquire a rules engine__.

- If an ID is used, the last saved state with that identifier will be loaded. If it does not exist yet, simply initialize void (execute the reset command).

- The state id is optional. Use `acquire_rules_engine()` or `acquire_rules_engine(None)` to get a rules engine without any internal state (jsut like executing a `reset` command).

In [7]:
# Get one instance of a rules engine
ID = "persistent_state_1"
re = rules_engine_pool.acquire_rules_engine(ID)

[DEBUG] 19-11-2019 11:11:16.432 - auracog_rules.rules_engines: Reading persisted state for id="persistent_state_1"
[WARNING] 19-11-2019 11:11:16.437 - auracog_rules.rules_engines_persistence: No persisted state found for id="persistent_state_1"


It is possible to have access to some lower level properties of the rules engine through `env`.

Execute `help(re.env)`for further details.

In [8]:
# Show currently asserted facts
list(re.env.facts())

[TemplateFact: f-0     (initial-fact)]

In [9]:
# Show currently defined rules
list(re.env.rules())

[Rule: (defrule MAIN::retract_initial_slot "This rule retracts the __initial_slot__ rule, needed for managing unique slots"
    ?f <- (slot __initial_slot__)
    =>
    (retract ?f)),
 Rule: (defrule MAIN::r_explore_1 "Do not suggest intents that have been just suggested"
    (now ?now)
    (suggested_intent (id ?id) (last_suggested ?last_suggested))
    (test (< (- ?now ?last_suggested) ?*MIN_SUGGESTION_TIME*))
    =>
    (assert (intent_suggestion ?id -1000 (gensym)))),
 Rule: (defrule MAIN::r_explore_2 "Explore on TV domain per day"
    (suggested_intent (id ?id) (num_suggested_day ?num_suggested))
    (test (in ?id ?*TV_DOMAIN_INTENTS*))
    (test (< ?num_suggested ?*MAX_SUGGESTIONS_DAY_TV*))
    =>
    (assert (intent_suggestion ?id 0.1 (gensym)))),
 Rule: (defrule MAIN::r_explore_3 "Explore on TV domain per week"
    (suggested_intent (id ?id) (num_suggested_week ?num_suggested))
    (test (in ?id ?*TV_DOMAIN_INTENTS*))
    (test (< ?num_suggested ?*MAX_SUGGESTIONS_WEEK_TV*))
   

__Start processing with the rules engine__

1. Set facts into the rules engine

In [10]:
# Reset the rules engine if needed
re.reset()

In [11]:
# This is the information for user AU123456
user_info_AU123456 = {
    "id": "AU123456", # User id
    "type": 1,        # Type of user (type of subscription)
    "cluster_id": 2,  # Id of the cluster corresponding to the user (from user profiling)
    "channel_id": "m-home", # Channel
    "at_home": True,  # User is at home
    "stb": True,      # STB is available
}

# This is the information on the previously suggested intents to user AU123456
suggested_intents_user_AU123456 = [
    [
        "suggested_intent", 
        {
            "user_id": "AU123456",
            "id": 222,
            "name": "tv.record",
            "num_requested_day": 0,
            "num_requested_week": 3,
            "num_suggested_day": 0,
            "num_suggested_week": 2,
            "num_selected_day": 1,
            "num_selected_week": 1,
            "last_suggested": time.time() - 2*60*60  # Last suggested 2 hours ago
        }
    ],
    [
        "suggested_intent", 
        {
            "user_id": "AU123456",
            "id": 191,
            "name": "tv.search",
            "num_requested_day": 2,
            "num_requested_week": 20,
            "num_suggested_day": 0,
            "num_suggested_week": 10,
            "num_selected_day": 2,
            "num_selected_week": 18,
            "last_suggested": time.time() - 5*60  # Last suggested 5 minutes ago
        }
    ],
    [
        "suggested_intent", 
        {
            "user_id": "AU123456",
            "id": 194,
            "name": "tv.search_similar",
            "num_requested_day": 0,
            "num_requested_week": 0,
            "num_suggested_day": 0,
            "num_suggested_week": 1,
            "num_selected_day": 0,
            "num_selected_week": 0,
            "last_suggested": time.time() - 3*24*60*60  # Last suggested 3 days ago
        }
    ],
    [
        "suggested_intent", 
        {
            "user_id": "AU123456",
            "id": 193,
            "name": "tv.question_time_loc",
            "num_requested_day": 0,
            "num_requested_week": 0,
            "num_suggested_day": 1,
            "num_suggested_week": 8,
            "num_selected_day": 0,
            "num_selected_week": 0,
            "last_suggested": time.time() - 24*60*60  # Last suggested 1 day ago
        }
    ]
]

# Current user session information
current_session = {
    "user_id": "AU123456",
    "intents": [191, 195]  # t.search, tv.display
}


In [12]:
re.set_facts([["user_info", user_info_AU123456]])
re.set_facts(suggested_intents_user_AU123456)
re.set_facts([["current_session", current_session]])

[DEBUG] 19-11-2019 11:11:16.643 - auracog_rules.rules_engines: Set facts into the rules engine in 1.219 ms
[DEBUG] 19-11-2019 11:11:16.652 - auracog_rules.rules_engines: Set facts into the rules engine in 7.534 ms
[DEBUG] 19-11-2019 11:11:16.656 - auracog_rules.rules_engines: Set facts into the rules engine in 0.205 ms


In [13]:
list(re.env.facts())

[TemplateFact: f-0     (initial-fact),
 ImpliedFact: f-1     (slot __initial_slot__),
 TemplateFact: f-2     (user_info (id "AU123456") (type 1) (cluster_id 2) (channel_id "m-home") (at_home TRUE) (stb TRUE)),
 TemplateFact: f-3     (suggested_intent (user_id "AU123456") (id 222) (name "tv.record") (num_requested_day 0) (num_requested_week 3) (num_suggested_day 0) (num_suggested_week 2) (num_selected_day 1) (num_selected_week 1) (last_suggested 1574151076.60553)),
 TemplateFact: f-4     (suggested_intent (user_id "AU123456") (id 191) (name "tv.search") (num_requested_day 2) (num_requested_week 20) (num_suggested_day 0) (num_suggested_week 10) (num_selected_day 2) (num_selected_week 18) (last_suggested 1574157976.60553)),
 TemplateFact: f-5     (suggested_intent (user_id "AU123456") (id 194) (name "tv.search_similar") (num_requested_day 0) (num_requested_week 0) (num_suggested_day 0) (num_suggested_week 1) (num_selected_day 0) (num_selected_week 0) (last_suggested 1573899076.60553)),
 T

__Reason__

In [14]:
re.reason()

[DEBUG] 19-11-2019 11:11:16.706 - auracog_rules.rules_engines: Rules engine _reason().self.env.run() took 0.344 ms
[DEBUG] 19-11-2019 11:11:16.708 - auracog_rules.rules_engines: Rules engine reason() took 1.593 ms. Facts asserted: 18. Rules defined: 7. Rules fired: 11


__Get results__

We are interested in the facts with name `intent_suggestion`.

In [15]:
intent_suggestions = re.collect_fact_values("intent_suggestion")
for s in intent_suggestions:
    print(s)

[DEBUG] 19-11-2019 11:11:16.725 - auracog_rules.rules_engines: Rules engine collect_fact_values() took 5.530 ms


[194, 0.5, 'gen1']
[222, 0.3, 'gen2']
[193, 0.5, 'gen3']
[193, 0.1, 'gen4']
[194, 0.1, 'gen5']
[194, 0.2, 'gen6']
[191, 0.1, 'gen7']
[222, 0.1, 'gen8']
[222, 0.2, 'gen9']
[191, -1000, 'gen10']


In [16]:
# Sum punctuations for each intent
res = {}
for s in intent_suggestions:
    p = res.get(s[0], 0)
    p += s[1]
    res[s[0]] = p
print(res)

{194: 0.8, 222: 0.6000000000000001, 193: 0.6, 191: -999.9}


__Release the rules engine__

In [17]:
# Release and do not save state
rules_engine_pool.release_rules_engine(re)

# If it is needed to sae the current state, execute
# rules_engine_pool.release_rules_engine(re, ID)